In [18]:
#dependencies
import pandas as pd
import json
from datetime import datetime
import re

In [19]:
#csvs from sql queries
kaddex = "swaps_kaddex.csv"
kdx = pd.read_csv(kaddex)

kdswap = "swaps_kdswap.csv"
kds = pd.read_csv(kdswap)

brodex = "swaps_bro-dex.csv"
bro = pd.read_csv(brodex)

In [20]:
def process(csv):
    df = pd.read_csv(csv)

    def process_tokens(row):
        try:
            if pd.notnull(row['result']):
                parsed = json.loads(row['result'])
                if isinstance(parsed.get("data"), list) and len(parsed["data"]) == 2:
                    t0, t1 = parsed["data"][0], parsed["data"][1]
                    amt0 = t0.get("amount", 0)
                    amt1 = t1.get("amount", 0)
                    if isinstance(amt0, dict): amt0 = float(amt0.get("decimal", 0))
                    if isinstance(amt1, dict): amt1 = float(amt1.get("decimal", 0))
                    return pd.Series([t0.get("token", ""), amt0, t1.get("token", ""), amt1])
        except Exception:
            pass

        try:
            if isinstance(row['code'], str) and 'swap-exact' in row['code']:
                numbers = re.findall(r'[-+]?\d*\.\d+|\d+', row['code'])
                token_matches = re.findall(r'\[(.*?)\]', row['code'])

                if len(numbers) >= 2 and token_matches:
                    token_list = token_matches[0].split()
                    if len(token_list) >= 2:
                        token0 = token_list[0].strip('",()')
                        token1 = token_list[1].strip('",()')
                        token0amount = float(numbers[0])
                        token1amount = float(numbers[1])
                        return pd.Series([token0, token0amount, token1, token1amount])
        except Exception:
            pass

        return pd.Series(["", 0, "", 0])

    df_tokens = df.apply(process_tokens, axis=1)
    df_tokens.columns = ['token0', 'token0amount', 'token1', 'token1amount']

    df['time'] = pd.to_datetime(df['creationtime'], unit='s', errors='coerce')
    df['requestkey'] = df['requestkey'].apply(
        lambda rk: f"https://explorer.kadena.io/mainnet/transaction/{rk}"
    )

    final_df = pd.concat([
        df[['sender', 'requestkey']],
        df_tokens,
        df['time']
    ], axis=1)

    cleaned_df = final_df[
        (final_df['token0'] != "") &
        (final_df['token1'] != "") &
        (final_df['token0amount'] != 0) &
        (final_df['token1amount'] != 0)
    ].copy()

    return cleaned_df

In [21]:
def process_brodex(csv):
    df = pd.read_csv(csv)

    successful = df[df['result'].str.contains('"status": "success"')].copy()

    def process_tokens_brodex(row):
        try:
            code = row['code']
            if 'bro-dex-wrapper' not in code or 'cancel-order' in code:
                return pd.Series(["", 0, "", 0])

            match = re.search(r'bro-dex-wrapper-([A-Z0-9]+)-([A-Z0-9]+)-M\.(buy|sell)', code)
            if not match:
                return pd.Series(["", 0, "", 0])
            tokenA, tokenB, side = match.groups()

            # Extract final two numbers: price and amount
            nums = re.findall(r'[-+]?\d*\.\d+|\d+', code)
            if len(nums) < 2:
                return pd.Series(["", 0, "", 0])
            price = float(nums[-2])
            amount = float(nums[-1])

            if side == 'buy':
                return pd.Series([tokenB, amount, tokenA, price])
            else:  # sell
                return pd.Series([tokenA, price, tokenB, amount])
        except Exception:
            return pd.Series(["", 0, "", 0])

    token_df = successful.apply(process_tokens_brodex, axis=1)
    token_df.columns = ['token0', 'token0amount', 'token1', 'token1amount']

    successful['time'] = pd.to_datetime(successful['creationtime'], unit='s', errors='coerce')
    successful['requestkey'] = successful['requestkey'].apply(
        lambda rk: f"https://explorer.kadena.io/mainnet/transaction/{rk}"
    )

    final_df = pd.concat([
        successful[['sender', 'requestkey']],
        token_df,
        successful['time']
    ], axis=1)

    cleaned_df = final_df[
        (final_df['token0'] != "") &
        (final_df['token1'] != "") &
        (final_df['token0amount'] > 0) &
        (final_df['token1amount'] > 0)
    ].copy()

    return cleaned_df

Mercatus

In [22]:
kaddex_df = process("swaps_kaddex.csv")
kaddex_df.to_csv("kdx_swaps.csv", index=False)

kaddex_df.head()

,sender,requestkey,token0,token0amount,token1,token1amount,time
0,k:b46e0140cf675d78c3b88953a5942c737e1f330ff700...,https://explorer.kadena.io/mainnet/transaction...,free.crankk01,4639.0,coin,3.463681e+01,2025-06-13 02:31:49
1,k:407bffab0d4eb3dc1c5ddfc7fca923fa4fb9a07be3f3...,https://explorer.kadena.io/mainnet/transaction...,free.crankk01,1000.0,coin,7.397144e+00,2025-06-13 02:32:38
2,k:407bffab0d4eb3dc1c5ddfc7fca923fa4fb9a07be3f3...,https://explorer.kadena.io/mainnet/transaction...,free.crankk01,1000.0,coin,7.372858e+00,2025-06-13 02:32:47
3,k:407bffab0d4eb3dc1c5ddfc7fca923fa4fb9a07be3f3...,https://explorer.kadena.io/mainnet/transaction...,free.crankk01,1000.0,coin,7.348692e+00,2025-06-13 02:32:56
4,k:fea29be0591af42646ca25dce0e356e77eae3ef8aff7...,https://explorer.kadena.io/mainnet/transaction...,coin,163.0,n_625e9938ae84bdb7d190f14fc283c7a6dfc15d58.ktoshi,1.375245e+09,2025-06-13 03:58:01


KDSwap

In [23]:
kdswap_df = process("swaps_kdswap.csv")
kdswap_df.to_csv("kds_swaps.csv", index=False)

kdswap_df.head()

,sender,requestkey,token0,token0amount,token1,token1amount,time
0,k:d665bcfe24cd76d6d531d01d32281c4d1ec4b0ad95d4...,https://explorer.kadena.io/mainnet/transaction...,n_582fed11af00dc626812cd7890bb88e72067f28c.bro,0.0149,coin,32.068918,2025-06-13 19:43:31
1,k:d665bcfe24cd76d6d531d01d32281c4d1ec4b0ad95d4...,https://explorer.kadena.io/mainnet/transaction...,coin,10.0000,n_518dfea5f0d2abe95cbcd8956eb97f3238e274a9.AZUKI,4339.061430,2025-06-13 14:24:54
2,k:d665bcfe24cd76d6d531d01d32281c4d1ec4b0ad95d4...,https://explorer.kadena.io/mainnet/transaction...,n_518dfea5f0d2abe95cbcd8956eb97f3238e274a9.AZUKI,4415.4181,coin,10.114271,2025-06-13 14:31:50
3,k:d665bcfe24cd76d6d531d01d32281c4d1ec4b0ad95d4...,https://explorer.kadena.io/mainnet/transaction...,coin,20.0000,kdlaunch.kdswap-token,9231.846460,2025-06-13 14:42:06
4,k:b46e0140cf675d78c3b88953a5942c737e1f330ff700...,https://explorer.kadena.io/mainnet/transaction...,free.crankk01,3209.0000,coin,23.924557,2025-06-13 02:31:49


Bro-Dex

In [24]:
brodex_df = process_brodex("swaps_bro-dex.csv")
brodex_df.to_csv("bro_swaps.csv",index=False)

brodex_df.head()

,sender,requestkey,token0,token0amount,token1,token1amount,time
15,k:7b9c8a048b89df9b3bc57d6e133381b094030c93fa3b...,https://explorer.kadena.io/mainnet/transaction...,BRO,0.0100,KDA,2.260949e+03,2025-06-13 04:16:30
16,k:7b9c8a048b89df9b3bc57d6e133381b094030c93fa3b...,https://explorer.kadena.io/mainnet/transaction...,BRO,0.0200,HERON,9.221481e+06,2025-06-13 04:16:33
19,k:7b9c8a048b89df9b3bc57d6e133381b094030c93fa3b...,https://explorer.kadena.io/mainnet/transaction...,BRO,0.0177,HERON,9.087505e+06,2025-06-13 04:16:33
20,k:7b9c8a048b89df9b3bc57d6e133381b094030c93fa3b...,https://explorer.kadena.io/mainnet/transaction...,KDA,2194.1212,BRO,9.110000e-03,2025-06-13 04:16:31
21,k:7b9c8a048b89df9b3bc57d6e133381b094030c93fa3b...,https://explorer.kadena.io/mainnet/transaction...,BRO,0.0100,KDA,2.294282e+03,2025-06-13 04:16:30
